In [1]:
# Импорт необходимых библиотек

import polars as pl
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
# Загрузка модели LLM
model_name = "IlyaGusev/saiga_gemma2_9b"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
model.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNorm((3584,), 

In [3]:
# Функция для подготовки промпта в формате Gemma-2
def prepare_prompt(messages):
    prompt = ""
    for message in messages:
        role = message["role"]
        content = message["content"]
        if role == "system":
            prompt += "<start_of_turn>system\n" + content + "<end_of_turn>\n"
        elif role == "user":
            prompt += "<start_of_turn>user\n" + content + "<end_of_turn>\n"
        elif role == "model":
            prompt += "<start_of_turn>model\n" + content
    return prompt


# Функция для генерации ответа модели
def generate_response(prompt, max_new_tokens=200):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
        )
    output = tokenizer.decode(output_ids[0][input_ids.shape[-1] :], skip_special_tokens=True)
    return output.strip()

In [8]:
def prepare_parameters_prompt_data(
    parameters_list: list[str],
    group_name: str,
    system_promt_group_parameters: str,
    user_promt_group_parameters: str,
) -> tuple[str]:
    user_promt_group_parameters_formatted = user_promt_group_parameters.format(group_name, "\n".join(parameters_list))

    return system_promt_group_parameters, user_promt_group_parameters_formatted


def process_message(text: str, system_prompt: str) -> str:
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": text}]

    prompt = prepare_prompt(messages)
    return generate_response(prompt)

In [9]:
mtr = pl.read_parquet("../MTR.parquet")

In [15]:
### ПРОМТ ПОЛУЧЕНИЯ ПАРАМЕТРОВ
system_promt_group_parameters = (
    """Ты — Сайга, русскоязычный ассистент. Ты помогаешь придумывать набор параметров для описания группы товаров"""
)
user_promt_group_parameters = """Выдели из описаний набор параметров, которые позволят единым образом описать товары из группы с названием {}.

ИНСТРУКЦИИ:
1. Каждый параметр должен характеризоваться 1 словом.
2. Набор параметров должен состоять не более чем из 10 параметров. 
3. Параметры должны основываться исключительно на информации из описаний.
4. Если описания короткие и неинформативные, ты можешь вернуть менее, чем 10 параметров.
5. Старайся понять, какие параметры отражают предоставленные описания товаров.
6. Возвращай набор параметров как название каждого отдельного параметра с ; в качестве разделителя между ними
7. Верни только набор параметров.

ВХОДНЫЕ ДАННЫЕ (ОПИСАНИЯ ТОВАРОВ):
Наименования и описания единиц товаров входящих в группу. Каждая пара будет начинаться с новой строчки и представлена в формате наименование товара: описание товара.
{}

ФОРМАТ ВЫВОДА:
параметр 1; параметр 2; параметр 3; параметр n

ПРИМЕР ВЫВОДА:
длина; ширина; высота; цвет
"""

In [17]:
system_promt_group_parameters_formatted, user_promt_group_parameters_formatted = prepare_parameters_prompt_data(
    mtr.head(4)["Параметры"].to_numpy(), "СОРОЧКА МУЖСКАЯ АО ФПК", system_promt_group_parameters, user_promt_group_parameters
)

In [18]:
process_message(user_promt_group_parameters_formatted, system_promt_group_parameters)

'model\nтип; пол; категория; группа; должность; галуна; звезды'